In [1]:
import time
from time import sleep

from random import randint

import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import *

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# transformando o arquivo excel em dataframe
#cartas_df = pd.read_excel('excel/lista_cartas_magic_com_edicao.xlsx')
cartas_df = pd.read_excel('excel/lista_completa_cartas_magic_com_edicao.xlsx')

# eliminando as duplicatas (cartas com o mesmo nome e edições diferentes)
cartas_df = cartas_df.drop_duplicates(
    subset='nome_portugues', keep="first").reset_index(drop=True)

In [3]:
cartas_df

,nome_portugues,nome_ingles,edicao
0,Abertura Temporal,Temporal Aperture,A Saga de Urza
1,Abraço de Shiv,Shiv's Embrace,A Saga de Urza
2,Agilidade,Agility,Miragem
3,Agulha de Arenito,Sandstone Needle,Máscara de Mercádia
4,Alma Desgarrada,Lost Soul,Sexta Edição Classica
...,...,...,...
280,Vorme Espinhoso,Spined Wurm,Fortaleza
281,Vulto da Masmorra,Dungeon Shade,Fortaleza
282,Xamã de Anaba,Anaba Shaman,Sexta Edição Classica
283,Zagaieiro de Elite,Elite Javelineer,Tempestade


In [4]:
# algumas configurações para o webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--proxy-server='direct://'")
chrome_options.add_argument("--proxy-bypass-list=*")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--ignore-certificate-errors')

# instanciando a versão do webdriver que será instalada na máquina local
service = Service(ChromeDriverManager().install())

# instanciando o webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Extração

In [5]:
nome_portugues_lista = []
nome_ingles_lista = []
edicao_lista = []
artista_lista = []
raridade_lista = []
valor_medio_srt_lista = []

cartas_web_dic = []

# efetuando busca no site
for portugues, ingles in zip(cartas_df['nome_portugues'], cartas_df['nome_ingles']):

    driver.get(f"https://www.ligamagic.com/?view=cards/card&card={portugues}&aux={ingles}")

    sleep(randint(2, 4))

    action = ActionChains(driver)

    # selecionar a edição
    edcard = 0

    while True:
        try:
            edicao_menu = driver.find_element(
                By.CSS_SELECTOR, f"#edcard_{edcard}> img:nth-child(1)")
            action.move_to_element(edicao_menu).click().perform()

        except NoSuchElementException:
            break
        else:
            edcard += 1

        # pegar o valor médio da carta
        nome_portugues = portugues
        nome_ingles = ingles
        edicao = driver.find_element(By.XPATH, '//*[@id="ed-nome"]/a').text
        artista = driver.find_element(By.XPATH, '//*[@id="ed-artista"]/a').text
        raridade = driver.find_element(
            By.XPATH, '//*[@id="ed-raridade"]/a').text
        valor_medio_srt = driver.find_element(
            By.XPATH, '//*[@id="card-info"]/div[5]/div[2]/div/div[4]').text

        # exibir nome e edição
        print(f"{edcard}. {nome_portugues} | {nome_ingles} | {edicao} | {artista} | {raridade} | {valor_medio_srt}")
        
        # criação do dicionário
        cartas_web_dic.append({
            'nome_portugues': nome_portugues,
            'nome_ingles': nome_ingles,
            'edicao': edicao,
            'artista': artista,
            'raridade': raridade,
            'valor_medio_srt': valor_medio_srt
        })

1. Abertura Temporal | Temporal Aperture | A Saga de Urza | Michael Sutfin | Rara | R$ 52,34
2. Abertura Temporal | Temporal Aperture | World Championship Decks | Michael Sutfin | Special | R$ 29,99
1. Abraço de Shiv | Shiv's Embrace | Jumpstart: Historic Horizons | Dave Kendall | Incomum | R$ 0,00
2. Abraço de Shiv | Shiv's Embrace | Magic 2014 | Dave Kendall | Incomum | R$ 0,46
3. Abraço de Shiv | Shiv's Embrace | Duel Decks: Knights vs. Dragons | Dave Kendall | Incomum | R$ 0,47
4. Abraço de Shiv | Shiv's Embrace | Magic 2011 | Dave Kendall | Incomum | R$ 0,53
5. Abraço de Shiv | Shiv's Embrace | A Saga de Urza | Bob Eggleton | Incomum | R$ 0,61
1. Agilidade | Agility | Miragem | Drew Tucker | Comum | R$ 0,36
1. Agulha de Arenito | Sandstone Needle | Máscara de Mercádia | Alan Rabinowitz | Comum | R$ 6,81
1. Alma Desgarrada | Lost Soul | Sexta Edição Classica | Randy Asplund-Faith | Comum | R$ 0,49
2. Alma Desgarrada | Lost Soul | Quinta Edição | Randy Asplund-Faith | Comum | R$ 0,4

In [6]:
# corversão do dicionário para dataframe
cartas_web_df = pd.DataFrame().from_dict(cartas_web_dic)

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt
0,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34"
1,Abertura Temporal,Temporal Aperture,World Championship Decks,Michael Sutfin,Special,"R$ 29,99"
2,Abraço de Shiv,Shiv's Embrace,Jumpstart: Historic Horizons,Dave Kendall,Incomum,"R$ 0,00"
3,Abraço de Shiv,Shiv's Embrace,Magic 2014,Dave Kendall,Incomum,"R$ 0,46"
4,Abraço de Shiv,Shiv's Embrace,Duel Decks: Knights vs. Dragons,Dave Kendall,Incomum,"R$ 0,47"
...,...,...,...,...,...,...
1295,Zumbis de Scathe,Scathe Zombies,Limited Edition Alpha,Jesper Myrfors,Comum,"R$ 208,13"
1296,Zumbis de Scathe,Scathe Zombies,Starter 1999,Tom Kyffin,Special,"R$ 1,87"
1297,Zumbis de Scathe,Scathe Zombies,International Collectors` Edition,Jesper Myrfors,Comum,"R$ 0,00"
1298,Zumbis de Scathe,Scathe Zombies,Collector`s Edition,Jesper Myrfors,Comum,"R$ 0,00"


In [7]:
# fechando o site
time.sleep(10)
driver.close()

# Transformação

In [8]:
# transformando o valor médio string para float
cartas_web_df['valor_medio'] = list(map(lambda x : x.split(" ")[1].replace(".", "").replace(",", "."), cartas_web_df['valor_medio_srt']))
cartas_web_df['valor_medio'] = list(map(lambda x : float(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio
0,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34
1,Abertura Temporal,Temporal Aperture,World Championship Decks,Michael Sutfin,Special,"R$ 29,99",29.99
2,Abraço de Shiv,Shiv's Embrace,Jumpstart: Historic Horizons,Dave Kendall,Incomum,"R$ 0,00",0.00
3,Abraço de Shiv,Shiv's Embrace,Magic 2014,Dave Kendall,Incomum,"R$ 0,46",0.46
4,Abraço de Shiv,Shiv's Embrace,Duel Decks: Knights vs. Dragons,Dave Kendall,Incomum,"R$ 0,47",0.47
...,...,...,...,...,...,...,...
1295,Zumbis de Scathe,Scathe Zombies,Limited Edition Alpha,Jesper Myrfors,Comum,"R$ 208,13",208.13
1296,Zumbis de Scathe,Scathe Zombies,Starter 1999,Tom Kyffin,Special,"R$ 1,87",1.87
1297,Zumbis de Scathe,Scathe Zombies,International Collectors` Edition,Jesper Myrfors,Comum,"R$ 0,00",0.00
1298,Zumbis de Scathe,Scathe Zombies,Collector`s Edition,Jesper Myrfors,Comum,"R$ 0,00",0.00


In [14]:
# calculando a comissão do mercado livre

def comissao_ml(valor_medio) -> int:
    ''' calculo de porcentagem (ml) e valor final do produto:
        - valores maiores ou iguais a R$ 79,00 a comissão do ML é de 11,5%
        - valores menores que R$ 79,00 a comissão do ML é de 11,5% + R$ 5,00
    '''
    if valor_medio >= 79:
        porcentagem_ml = (0.115 * valor_medio)
    else:
        porcentagem_ml = (0.115 * valor_medio) + 5

    valor_ml = round((porcentagem_ml + valor_medio), 2)
    
    return valor_ml


cartas_web_df['valor_ml'] = list(map(lambda x: comissao_ml(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34,63.36
1,Abertura Temporal,Temporal Aperture,World Championship Decks,Michael Sutfin,Special,"R$ 29,99",29.99,38.44
2,Abraço de Shiv,Shiv's Embrace,Jumpstart: Historic Horizons,Dave Kendall,Incomum,"R$ 0,00",0.00,5.00
3,Abraço de Shiv,Shiv's Embrace,Magic 2014,Dave Kendall,Incomum,"R$ 0,46",0.46,5.51
4,Abraço de Shiv,Shiv's Embrace,Duel Decks: Knights vs. Dragons,Dave Kendall,Incomum,"R$ 0,47",0.47,5.52
...,...,...,...,...,...,...,...,...
1295,Zumbis de Scathe,Scathe Zombies,Limited Edition Alpha,Jesper Myrfors,Comum,"R$ 208,13",208.13,232.06
1296,Zumbis de Scathe,Scathe Zombies,Starter 1999,Tom Kyffin,Special,"R$ 1,87",1.87,7.09
1297,Zumbis de Scathe,Scathe Zombies,International Collectors` Edition,Jesper Myrfors,Comum,"R$ 0,00",0.00,5.00
1298,Zumbis de Scathe,Scathe Zombies,Collector`s Edition,Jesper Myrfors,Comum,"R$ 0,00",0.00,5.00


In [15]:
# interseção das duas tabelas (para retornar apenas as edições de interece)
cartas_final_df = pd.merge(cartas_df, cartas_web_df, how='left', on=(
    'nome_portugues', 'nome_ingles', 'edicao'))

cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34,63.36
1,Abraço de Shiv,Shiv's Embrace,A Saga de Urza,Bob Eggleton,Incomum,"R$ 0,61",0.61,5.68
2,Agilidade,Agility,Miragem,Drew Tucker,Comum,"R$ 0,36",0.36,5.40
3,Agulha de Arenito,Sandstone Needle,Máscara de Mercádia,Alan Rabinowitz,Comum,"R$ 6,81",6.81,12.59
4,Alma Desgarrada,Lost Soul,Sexta Edição Classica,Randy Asplund-Faith,Comum,"R$ 0,49",0.49,5.55
...,...,...,...,...,...,...,...,...
280,Vorme Espinhoso,Spined Wurm,Fortaleza,Keith Parkinson,Comum,"R$ 0,40",0.40,5.45
281,Vulto da Masmorra,Dungeon Shade,Fortaleza,Jason Alexander Behnke,Comum,"R$ 0,36",0.36,5.40
282,Xamã de Anaba,Anaba Shaman,Sexta Edição Classica,Anson Maddocks,Comum,"R$ 0,38",0.38,5.42
283,Zagaieiro de Elite,Elite Javelineer,Tempestade,Mark Poole,Comum,"R$ 0,37",0.37,5.41


In [16]:
# teste para saber se foi realmente retoranda a lista desejada
if len(cartas_df["nome_portugues"]) == len(cartas_final_df["nome_portugues"]):
    print('Ok!')
else:
    print('Deu errado!')

Ok!


# Carregamento

In [18]:
# geração do arquivo final
cartas_final_df.to_excel("excel/cartas_magic_output.xlsx")
cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34,63.36
1,Abraço de Shiv,Shiv's Embrace,A Saga de Urza,Bob Eggleton,Incomum,"R$ 0,61",0.61,5.68
2,Agilidade,Agility,Miragem,Drew Tucker,Comum,"R$ 0,36",0.36,5.40
3,Agulha de Arenito,Sandstone Needle,Máscara de Mercádia,Alan Rabinowitz,Comum,"R$ 6,81",6.81,12.59
4,Alma Desgarrada,Lost Soul,Sexta Edição Classica,Randy Asplund-Faith,Comum,"R$ 0,49",0.49,5.55
...,...,...,...,...,...,...,...,...
280,Vorme Espinhoso,Spined Wurm,Fortaleza,Keith Parkinson,Comum,"R$ 0,40",0.40,5.45
281,Vulto da Masmorra,Dungeon Shade,Fortaleza,Jason Alexander Behnke,Comum,"R$ 0,36",0.36,5.40
282,Xamã de Anaba,Anaba Shaman,Sexta Edição Classica,Anson Maddocks,Comum,"R$ 0,38",0.38,5.42
283,Zagaieiro de Elite,Elite Javelineer,Tempestade,Mark Poole,Comum,"R$ 0,37",0.37,5.41
